In [2]:
#TO-DO-1
import json
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    #print(json_data)

import os
os.environ['SENDER_EMAIL'] = json_data['SENDER_EMAIL'] ## INPUT: Your Email
os.environ['PWD_EMAIL'] = json_data['PWD_EMAIL'] ## INPUT: Your App Password
os.environ['WEATHER_API_KEY'] = json_data['WEATHER_API_KEY'] ## INPUT: OpenWeather API Key
print('DONE! Get Env. Variable')

import email, smtplib, ssl # to automate email
import email as mail
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import requests, json # to pull API, and work with json
import datetime as dt # to work with date, time
from bs4 import BeautifulSoup # to work with web scrapping (HTML)
import pandas as pd # to work with tables (DataFrames) data
from IPython.core.display import HTML # to display HTML in the notebook


DONE! Get Env. Variable


In [40]:
print(os.environ['SENDER_EMAIL'])

shoppingwithlu2020@gmail.com


### TODO #2
1.   Từ Email Bot Code viết function `send_email`:
  - **inputs**: subject, receiver_email, part1, part2, part3
  - **output**: thực hiện việc gửi email và in ra kết quả
2.   Kết hợp Loops (For ... in) với function để gửi cùng nội dung email đến 1 list user: `email_list = ['abc@gmail.com', 'xyz@gmail.com']` (thay đổi email bằng các email thật của bạn để test)

In [3]:
def send_email(receiver_email, subject, part1, part2, part3):
  # YOUR CODE
    sender_email = os.environ['SENDER_EMAIL']
    password = os.environ['PWD_EMAIL'] 

    # (1) Create the email head (sender, receiver, and subject)
    email = MIMEMultipart()
    email["From"] = sender_email
    email["To"] = receiver_email 
    email["Subject"] = subject
    #print(subject)

    # (2) Create Body part
    # We use html, you can convert word to html: https://wordtohtml.net/
    html1 = """
        <html>
        <h1><strong>Hello World</strong></h1>
        <body>
        <p>Hi!<br>
          How are you?<br>
          Here is the <a href="https://docs.python.org/3.4/library/email-examples.html">link</a> you wanted.
        </p>
        </body>
        </html>
        """
    html2 = """
    <html>
    Email sent at <b>{}</b><br>
    </html>
    """.format(dt.datetime.now().isoformat())

    text3 = '--- End ----'

    # Combine parts
    part1 = MIMEText(html1, 'html')
    part2 = MIMEText(html2, 'html')
    part3 = MIMEText(part3, 'html')

    email.attach(part1)
    email.attach(part2)
    email.attach(part3)

    # (3) Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_email, password) #login with mail_id and password
    text = email.as_string()
    session.sendmail(sender_email, receiver_email, text)
    session.quit()

    print('DONE! Mail Sent'.format(sender_email, receiver_email))

In [340]:
email_list = ['lovelylife.158@gmail.com','shoppingwithlu2020@gmail.com']
for i in email_list:
  # YOUR CODE
  send_email(i,'Myemail', part1, part2, "Have a nice week")

DONE! Mail Sent
DONE! Mail Sent


### TODO #3
1. Trên file json kết quả:
  - Lấy thông tin trạng thái thời tiết (weather) chính (main) ở London
  - Lấy lat & lon (kinh độ và vĩ độ của London)
2. Viết function `get_weather_data` 
  - inputs: city
  - outputs: đoạn text thông tin thời tiết
3. Dùng function `send_email` in **TODO #1**, thay phần 3 trong nội dùng email bằng thông tin thời tiết của London gửi cho 1 địa chỉ email của bạn

In [342]:
# YOUR CODE
print(data['weather'][0]['main']) #thời tiết (weather) chính (main) ở London
print(data['coord']['lat']) #Lấy lat & lon (kinh độ và vĩ độ của London)
print(data['coord']['lon'])


Clouds
51.5085
-0.1257


In [10]:
def get_weather_data(city):
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"    # City Name 
    CITY = city    # API key 
    API_KEY = os.environ['WEATHER_API_KEY']    # upadting the URL
    URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY    # HTTP request
    response = requests.get(URL)    # checking the status code of the request
    weather = 'to be update'
    if response.status_code == 200:      # getting data in the json format
      data = response.json()      # getting the main dict block
      main = data['main']      # getting temperature
      temperature = str(main['temp'])      # getting the humidity
      humidity = str(main['humidity'])      # getting the pressure
      pressure = str(main['pressure'])      # weather report
      report = data['name']
      weather = 'City: '+data['name']+'<br>Temperature: '+ temperature +'<br>Humidity: '+humidity+'<br>Pressure: '+pressure+'<br>Weather Report: '+data['weather'][0]['main']
    else:      # showing the error message
      print("Error in the HTTP request")
      print(response)
    return weather
    

from IPython.core.display import HTML
HTML(get_weather_data('Hanoi'))


In [156]:
send_email('lovelylife.158@gmail.com',"To-do-3",part1,part2,get_weather_data('London'))

DONE! Mail Sent


### TODO #4
1. Dùng function `send_email` in Excercise 1.1, thay phần 3 trong nội dùng email bằng thông tin COVID-19 từ Wikipedia gửi cho 1 địa chỉ email của bạn

In [343]:
def get_news(articlelink): #funtion for getting news from link in wikipedia
    response = requests.get(
        url=articlelink,
    )
    soup = BeautifulSoup(response.content, 'html.parser')
    p = soup.find(id="bodyContent").find_all("p") #find paragraph
    for i in range(len(p)): #find 1st paragraph more than 5 word
        if len(p[i])>5:
            k = i
            break
    title = soup.find(id="firstHeading")
    Ref= "Click here "+ "<a href="+articlelink+">"+title.string+"</a>" +" for full article"
    return str(p[k])+Ref


In [354]:
HTML(get_news("https://en.wikipedia.org/wiki/China")) #example of funtion results

In [358]:
response = requests.get(url="https://blog.khanacademy.org/khan-academy-now-available-in-50-languages/",)
print("API Status Code: "+ str(response.status_code))
soup = BeautifulSoup(response.content, 'html.parser')
print(response.content)
#test with other link but can not get

API Status Code: 403
b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'


In [321]:
send_email('lovelylife.158@gmail.com',"To-do-4",part1,part2,get_news("https://en.wikipedia.org/wiki/COVID-19"))

DONE! Mail Sent


In [329]:
csv = """email,contents,city
lovelylife.158@gmail.com,News,London
shoppingwithlu2020@gmail.com,Weather,Hanoi
"""
f = open('email_list.csv','w') ## write to text file
f.write(csv)
f.close()

In [330]:
!dir

 Volume in drive D is DATA
 Volume Serial Number is BA7D-8B78

 Directory of d:\Github\atom-assignments\assignment_2

05/15/2021  09:25 AM    <DIR>          .
05/15/2021  09:25 AM    <DIR>          ..
05/15/2021  03:08 AM         1,454,447 ATOM_Home_Assignment_2.ipynb
05/16/2021  10:41 PM               103 email_list.csv
05/15/2021  02:00 AM               151 env_variable.json
05/15/2021  02:29 AM               116 weather.txt
               4 File(s)      1,454,817 bytes
               2 Dir(s)  87,857,238,016 bytes free


In [331]:
import pandas as pd
email_list = pd.read_csv('email_list.csv')

In [332]:
email_list

,email,contents,city
0,lovelylife.158@gmail.com,News,London
1,shoppingwithlu2020@gmail.com,Weather,Hanoi


### TODO #5
* Add các địa chỉ email có ý nghĩa, và thêm dòng vào file csv (lưu ý không có dấu cách)
* Dùng code để gửi email cho một list từ csv:
  - Nếu email có contents = 'News' => Gửi thông tin COVID19
  - Nếu email có contents = 'Weather' => Gửi thông tin thời tiết theo thành phố trong cột city (lưu ý check các city được hỗ trợ trong API)

**Hints:**
* Sử dụng loops for ... in 
* Sử dụng function `send_email`, `get_weather_data`

In [336]:
for i in list(range(2)):
  print('Email #{}'.format(i))
  if email_list['contents'][i] == 'News':
    print('COVID To: {}'.format(email_list['email'][i]))
    # TODO: Send a COVID News
    send_email(email_list['email'][i],"News for new day",part1,part2,get_news("https://en.wikipedia.org/wiki/COVID-19"))
  elif email_list['contents'][i] == 'Weather':
    print('Weather To: {}'.format(email_list['email'][i]))
    # TODO: Send a Weather updates by the city
    send_email(email_list['email'][i],"Good morning",part1,part2,get_weather_data(email_list['city'][i]))
  else:
    print('Invalid Contents')
print('DONE!')

Email #0
COVID To: lovelylife.158@gmail.com
DONE! Mail Sent
Email #1
Weather To: shoppingwithlu2020@gmail.com
DONE! Mail Sent
DONE!
